In [36]:
import os
import sys

import more_itertools
import torch
import transformers

import general_utils


In [5]:
model_name = "google/flan-t5-xl"
t = transformers.AutoTokenizer.from_pretrained(model_name)

m = transformers.T5ForConditionalGeneration.from_pretrained(
    model_name, torch_dtype=torch.bfloat16
)
m.parallelize()


In [20]:
def prep_q(text):
    return {
        k: v.to(m.device) for k, v in t(text, return_tensors="pt", padding=True).items()
    }

def prep_a(q, gen_flags=None):
    gen_flags = gen_flags or {}
    return [t.decode(x) for x in m.generate(**q, **gen_flags)]


In [38]:
q = prep_q(
    "Question: Mary and John got married last weekend. There were 20 private cars and 12 buses parked outside the church. "
    "After the ceremony, each bus carried 35 people and each car carried 3 people. How many people were inside the church? Let's think about it step by step. Chain of thought:",
)

general_utils.print_list(prep_a(q, dict(max_new_tokens=200, num_beams=6, num_return_sequences=6)))

- <pad> First find the total number of people in the buses: 35 people / bus * 12 buses = 375 people. Then 
find the total number of people in the cars: 3 people / car * 20 cars = 60 people. Then add the number of people in
each group to find the total number of people: 375 people + 60 people = 850 people. Then multiply the number of 
people in the church by the number of people in each group to find the total number of people: 850 people / group *
2 people / group = 1200 people. Therefore the answer is 1200.</s>

- <pad> First find the total number of people in the buses: 35 people / bus * 12 buses = 375 people. Then 
find the total number of people in the cars: 3 people / car * 20 cars = 60 people. Then add the number of people in
each group to find the total number of people: 375 people + 60 people = 850 people. Then multiply the number of 
people in the church by the number of people in each group to find the total number of people: 850 people / group *
2 people / group = 3000 people. Therefore the answer is 3000.</s>

- <pad> First find the total number of people in the buses: 35 people / bus * 12 buses = 375 people. Then 
find the total number of people in the cars: 3 people / car * 20 cars = 60 people. Then add the number of people in
each group to find the total number of people: 375 people + 60 people = 850 people. Then multiply the number of 
people in the church by the number of people in each group to find the total number of people: 850 people / group *
2 people / group = 1800 people. Therefore, the answer is 1800.</s><pad>

- <pad> First find the total number of people in the buses: 35 people / bus * 12 buses = 375 people. Then 
find the total number of people in the cars: 3 people / car * 20 cars = 60 people. Then add the number of people in
each group to find the total number of people: 375 people + 60 people = 850 people. Then multiply the number of 
people in the church by the number of people in each group to find the total number of people: 850 people / group *
2 people / group = 2700 people. Therefore the answer is 2700.</s>

- <pad> First find the total number of people in the buses: 35 people / bus * 12 buses = 375 people. Then 
find the total number of people in the cars: 3 people / car * 20 cars = 60 people. Then add the number of people in
each group to find the total number of people: 375 people + 60 people = 850 people. Then multiply the number of 
people in the church by the number of people in each group to find the total number of people: 850 people / group *
2 people / group = 1350 people. Therefore the answer is 1350.</s>

- <pad> First find the total number of people in the buses: 35 people / bus * 12 buses = 375 people. Then 
find the total number of people in the cars: 3 people / car * 20 cars = 60 people. Then add the number of people in
each group to find the total number of people: 375 people + 60 people = 850 people. Then multiply the number of 
people in the church by the number of people in each group to find the total number of people: 850 people / group *
2 people / group = 1800 people. Therefore the answer is 1800.</s><pad><pad>

In [39]:
20 * 2 + 12 * 35

424